In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import os
!pip install openpyxl

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
file_name = "/content/drive/MyDrive/AI_PYTHON/naive2.xlsx"
df = pd.read_excel(file_name,index_col=0)
print(df)

     color    type    origin stolen?
no                                  
1      red  sports  domestic     yes
2      red  sports  domestic      no
3      red  sports  domestic     yes
4   yellow  sports  domestic      no
5   yellow  sports  imported     yes
6   yellow     suv  imported      no
7   yellow     suv  imported     yes
8   yellow     suv  domestic      no
9      red     suv  imported      no
10     red  sports  imported     yes


In [5]:
X, y=df.drop(["stolen?"],axis=1),df["stolen?"]
print(X.shape)
print(y)

(10, 3)
no
1     yes
2      no
3     yes
4      no
5     yes
6      no
7     yes
8      no
9      no
10    yes
Name: stolen?, dtype: object


In [6]:
def accuracy_score(y_true, y_pred):

	"""	score = (y_true - y_pred) / len(y_true) """

	return round(float(sum(y_pred == y_true))/float(len(y_true)) * 100 ,2)

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score

class NaiveBayes:
    """
    Bayes Theorem:
                                  Likelihood * Class prior probability
        Posterior Probability = -------------------------------------
                                      Predictor prior probability

                                   P(x|c) * P(c)
                          P(c|x) = ------------------
                                              P(x)
    """

    def __init__(self):
        """
        Attributes:
            likelihoods: Likelihood of each feature per class
            class_priors: Prior probabilities of classes
            pred_priors: Prior probabilities of features
            features: All features of the dataset
        """
        self.features = None
        self.likelihoods = {}
        self.class_priors = {}
        self.pred_priors = {}

        self.X_train = None
        self.y_train = None
        self.train_size = 0
        self.num_feats = 0

    def fit(self, X, y):
        """
        Fit the Naive Bayes model to the training data.
        """
        self.features = list(X.columns)
        self.X_train = X
        self.y_train = y
        self.train_size = X.shape[0]
        self.num_feats = X.shape[1]

        for feature in self.features:
            self.likelihoods[feature] = {}
            self.pred_priors[feature] = {}

            for feat_val in np.unique(self.X_train[feature]):
                self.pred_priors[feature].update({feat_val: 0})

                for outcome in np.unique(self.y_train):
                    # Convert feat_val and outcome to strings explicitly before concatenation
                    self.likelihoods[feature].update({str(feat_val) + '_' + str(outcome): 0})
                    self.class_priors.update({outcome: 0})

        self._calc_class_prior()
        self._calc_likelihoods()
        self._calc_predictor_prior()

    def _calc_class_prior(self):
        """ P(c) - Prior Class Probability """
        for outcome in np.unique(self.y_train):
            outcome_count = sum(self.y_train == outcome)
            self.class_priors[outcome] = outcome_count / self.train_size

    def _calc_likelihoods(self):
        """ P(x|c) - Likelihood """
        for feature in self.features:
            for outcome in np.unique(self.y_train):
                outcome_count = sum(self.y_train == outcome)
                feat_likelihood = self.X_train[feature][self.y_train[self.y_train == outcome].index.values.tolist()].value_counts().to_dict()

                for feat_val, count in feat_likelihood.items():
                    # Convert feat_val and outcome to strings explicitly before concatenation
                    self.likelihoods[feature][str(feat_val) + '_' + str(outcome)] = count / outcome_count

    def _calc_predictor_prior(self):
        """ P(x) - Evidence """
        for feature in self.features:
            feat_val_counts = self.X_train[feature].value_counts().to_dict()
            for feat_val, count in feat_val_counts.items():
                self.pred_priors[feature][feat_val] = count / self.train_size

    def predict(self, X):
        """ Calculates Posterior probability P(c|x) """
        results = []
        X = np.array(X)

        for query in X:
            probs_outcome = {}
            for outcome in np.unique(self.y_train):
                prior = self.class_priors[outcome]
                likelihood = 1

                for feat, feat_val in zip(self.features, query):
                    likelihood *= self.likelihoods[feat][str(feat_val) + '_' + str(outcome)]

                posterior = likelihood * prior
                probs_outcome[outcome] = posterior

            result = max(probs_outcome, key=lambda x: probs_outcome[x])
            print(probs_outcome)
            results.append(result)

        return np.array(results)

In [8]:
nb_clf = NaiveBayes()
nb_clf.fit(X, y)

print("Train Accuracy: {}".format(accuracy_score(y, nb_clf.predict(X))))

#Query 1:
# color type origin stolen?
query = np.array([['red','suv','domestic']])
print("Query:- {} ---> {}".format(query, nb_clf.predict(query)))

{'no': 0.04800000000000001, 'yes': 0.096}
{'no': 0.04800000000000001, 'yes': 0.096}
{'no': 0.04800000000000001, 'yes': 0.096}
{'no': 0.072, 'yes': 0.06400000000000002}
{'no': 0.048, 'yes': 0.09600000000000002}
{'no': 0.072, 'yes': 0.024000000000000004}
{'no': 0.072, 'yes': 0.024000000000000004}
{'no': 0.108, 'yes': 0.016000000000000004}
{'no': 0.048, 'yes': 0.036}
{'no': 0.03200000000000001, 'yes': 0.144}
Train Accuracy: 0.8
{'no': 0.072, 'yes': 0.024}
Query:- [['red' 'suv' 'domestic']] ---> ['no']
